In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 30.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel


In [3]:
torch.manual_seed(42)

In [4]:
import random
random.seed(1234)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('flax-community/gpt2-medium-persian', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('flax-community/gpt2-medium-persian').cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50001, 1024)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_json("/content/drive/MyDrive/NLP/Project/datasets.json")

In [ ]:
def make_trait(row):
    row["trait_0"] = row["mbti_result"][0]
    row["trait_1"] = row["mbti_result"][1]
    row["trait_2"] = row["mbti_result"][2]
    row["trait_3"] = row["mbti_result"][3]
    return row

df = df.apply(make_trait, axis=1)

In [ ]:
class PersonalityDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
for trait in range(1):
    curr_label_str = f"trait_{trait}"
    curr_df = df[["tweets", curr_label_str]]
    grouped_curr_df = curr_df.groupby(curr_label_str)
    for label, frame in grouped_curr_df:
      if label == "I":
        continue
      print(f"training model on trait {label} ...")
      top_k = 30
      max_length = 750
      tweets = frame["tweets"].apply(lambda x : " ".join(random.sample(x, top_k))[:max_length])
      max_length = max([len(tokenizer.encode(tweet)) for tweet in tweets])
      dataset = PersonalityDataset(tweets, tokenizer, max_length=max_length)
      train_size = int(0.9 * len(dataset))
      train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
      training_args = TrainingArguments(output_dir='./models', num_train_epochs=2, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')
      Trainer(model=model,  args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()


training model on trait E ...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,6.223100
200,2.636500
300,2.437500
400,2.365600
500,2.340300
600,2.308200
700,2.280900
800,2.303900
900,2.266600
1000,2.260500


In [ ]:
model.save_pretrained("/content/drive/MyDrive/NLP/E.language_model")

In [ ]:
load_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/NLP/E.language_model")

In [ ]:
generated = tokenizer("<|startoftext|> ", return_tensors="pt").input_ids.cuda()

load_model = load_model.cuda()

sample_outputs = load_model.generate(generated, do_sample=True, top_k=50,
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)

with open("/content/drive/MyDrive/NLP/E_example.txt", "a+", encoding="utf-8") as f:
  for i, sample_output in enumerate(sample_outputs):
      f.write("{}: {}\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.
